In [3]:
import pandas as pd
from impala.dbapi import connect
from impala.util import as_pandas
import ast
import math

conn = connect(host='salest-master-server', port=21050)
cur = conn.cursor()

# daily transaction agg
cur.execute('USE salest')
cur.execute("""
        SELECT year_month, SUM(num_of_product) AS num_of_product, SUM(sales_amount) AS total_amount
        FROM (
            SELECT SUBSTR(date_receipt_num,1,7) AS year_month, num_of_product, sales_amount
            FROM ext_tr_receipt WHERE SUBSTR(date_receipt_num,1,4) = '2015'
        ) view_tr_recipt
        GROUP BY year_month ORDER BY year_month ASC
        """
)

df = as_pandas(cur)
conn.close()

year = 2015
unit_numofproduct = 1
unit_totalamount = 10000

month_index_arr = []

for month in range(1,13):
    month_index_arr.append("{0}-{1:02d}".format(year,month))
    
df_base_index = pd.DataFrame(data=month_index_arr, columns=['year_month'])
df_all_monatly_sales_volume = pd.merge(df, df_base_index, on='year_month', how='outer').fillna(0).sort_values(by='year_month',ascending='1')

df_list = list(df_all_monatly_sales_volume.itertuples(index=False))
df_column_name_list = list(df.columns.values)

list_month_sales_volume = []
dict_month_sales_volume = {}

for row in df_list:
    dict_month_sales_volume = {}
        
    for key,value in zip(df_column_name_list, row):
        if(key=='num_of_product'):
            value = int(round(value / unit_numofproduct))
        if(key=='total_amount'):
            value = int(round(value / unit_totalamount))
        dict_month_sales_volume[key] = value
        
    list_month_sales_volume.append(dict_month_sales_volume.copy())

In [33]:
list_month_sales_volume

[{'num_of_product': 1621, 'total_amount': 450, 'year_month': '2015-01'},
 {'num_of_product': 1582, 'total_amount': 450, 'year_month': '2015-02'},
 {'num_of_product': 1747, 'total_amount': 505, 'year_month': '2015-03'},
 {'num_of_product': 1317, 'total_amount': 379, 'year_month': '2015-04'},
 {'num_of_product': 2585, 'total_amount': 651, 'year_month': '2015-05'},
 {'num_of_product': 2091, 'total_amount': 574, 'year_month': '2015-06'},
 {'num_of_product': 2335, 'total_amount': 671, 'year_month': '2015-07'},
 {'num_of_product': 2984, 'total_amount': 855, 'year_month': '2015-08'},
 {'num_of_product': 1997, 'total_amount': 598, 'year_month': '2015-09'},
 {'num_of_product': 1889, 'total_amount': 537, 'year_month': '2015-10'},
 {'num_of_product': 1291, 'total_amount': 391, 'year_month': '2015-11'},
 {'num_of_product': 1343, 'total_amount': 395, 'year_month': '2015-12'}]

In [8]:
df_tr_receipt_menumap = as_pandas(cur)
conn.close()

conn = connect(host='salest-master-server', port=21050)
cur = conn.cursor()

# daily transaction agg
cur.execute('USE salest')
cur.execute(
    """
        SELECT SUBSTR(ext_tr_receipt.date_receipt_num,1,7) AS year_month, 
              ext_tr_receipt.num_of_product, ext_tr_receipt.sales_amount AS total_amount,
            ext_menumap_info.product_name, ext_menumap_info.cate_name, ext_menumap_info.price
        FROM ext_tr_receipt JOIN ext_menumap_info USING (product_code) 
        WHERE SUBSTR(date_receipt_num,1,4) = '2014'"""
)

df_tr_receipt_menumap = as_pandas(cur)
conn.close()

In [2]:
import pandas as pd
from impala.dbapi import connect
from impala.util import as_pandas
import ast
import math

conn = connect(host='salest-master-server', port=21050)
cur = conn.cursor()

# daily transaction agg
cur.execute('USE salest')
cur.execute(
    """
        SELECT year_month_day, SUM(num_of_product) AS num_of_product, SUM(sales_amount) AS total_amount
        FROM (
            SELECT SUBSTR(date_receipt_num,1,10) AS year_month_day, num_of_product, sales_amount
            FROM ext_tr_receipt
        ) view_tr_recipt
        GROUP BY year_month_day ORDER BY year_month_day ASC"""
)

df = as_pandas(cur)
conn.close()

ERROR: An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 4))



OperationalError: Operation is in ERROR_STATE

In [86]:
import pandas as pd
from impala.dbapi import connect
from impala.util import as_pandas
import ast
import math
from pandas import  MultiIndex, Series

year = '2014'

conn = connect(host='salest-master-server', port=21050)
cur = conn.cursor()

cur.execute('USE salest')
cur.execute(
    """
        SELECT SUBSTR(ext_tr_receipt.date_receipt_num,1,7) AS year_month, 
              ext_tr_receipt.num_of_product, ext_tr_receipt.sales_amount AS total_amount,
            ext_menumap_info.product_name, ext_menumap_info.cate_name, ext_menumap_info.price
        FROM ext_tr_receipt JOIN ext_menumap_info USING (product_code) WHERE SUBSTR(date_receipt_num,1,4) = '2014'"""
    )

df_tr_receipt_menumap = as_pandas(cur)
conn.close()

def aggregation(row):
    total_amount = row['total_amount'].sum()
    return pd.Series([total_amount], index=['total_amount'])
    
df_monthly_product_tr = df_tr_receipt_menumap.groupby(['year_month','cate_name']).apply(aggregation)

def gen_dict_total_amount(month_rows):
    monthlyDict = {}
    monthlyDictKey = month_rows.index.get_level_values('year_month')[0]
        
    monthCateItemsStr = "{"
    for item in zip(month_rows.index.get_level_values('cate_name'),month_rows['total_amount']):
        monthCateItemsStr += "'{0}':{1},".format(item[0],item[1]);
        
    monthCateItemsStr = monthCateItemsStr[:-1]
    monthCateItemsStr += "}"
        
    monthlyDict = ast.literal_eval(monthCateItemsStr)
    monthlyDict['year_month'] = month_rows.index.get_level_values('year_month')[0]

    return monthlyDict


#df_all_monatly_sales_volume = pd.merge(df_base_index, df_monthly_product_tr, left_on='year_month', right_index='True', how='outer')
#.fillna(0).sort_values(by='year_month',ascending='1')
###
 
    
#mothlyTotalAmountDictItems = df_monthly_product_tr.groupby(df_monthly_product_tr.index.get_level_values('year_month')).apply(gen_dict_total_amount)

#mothlyTotalAmountDict = {}
#mothlyTotalAmountList = []
#for item in mothlyTotalAmountDictItems:
#    mothlyTotalAmountList.append(item)
#    mothlyTotalAmountDict['total_amount'] = mothlyTotalAmountList


In [1]:
def genDefaultMontlyCateTotalAmountDataFrame(df_monthly_product_tr,year):
    unique_count_of_category = len(df_monthly_product_tr.index.get_level_values(1).unique())

    month_index_arr = []
    cate_index_arr = []

    for month in range(1,13):
        for cate in range(unique_count_of_category):
            month_index_arr.append("{0}-{1:02d}".format(year,month))

    unique_cate_index_arr = df_monthly_product_tr.ix[0:unique_count_of_category].index.get_level_values(1)

    for month in range(1,13):
        for cate in unique_cate_index_arr:
            cate_index_arr.append(cate)

    full_month_cate_multi_index = pd.MultiIndex.from_tuples(zip(month_index_arr, cate_index_arr), names=['year_month', 'cate_name'])
    df_full_month_cate_default = pd.DataFrame(0, index=full_month_cate_multi_index, columns=['total_amount'])
    return df_full_month_cate_default


def agg_montly_total_amount_by_product_cate(year):
    
    conn = connect(host='salest-master-server', port=21050)
    cur = conn.cursor()

    cur.execute('USE salest')
    cur.execute(
    """
        SELECT SUBSTR(view_tr_receipt.date_receipt_num,1,7) AS year_month, 
              view_tr_receipt.num_of_product, view_tr_receipt.sales_amount AS total_amount,
            ext_menumap_info.product_name, ext_menumap_info.cate_name, ext_menumap_info.price
        FROM (SELECT * FROM ext_tr_receipt WHERE SUBSTR(date_receipt_num,1,4) = '""" + year + "'" +
    """) view_tr_receipt JOIN ext_menumap_info USING (product_code)"""
    )

    df_tr_receipt_menumap = as_pandas(cur)
    conn.close()
    
    def aggregation(row):
        total_amount = row['total_amount'].sum()
        return pd.Series([total_amount], index=['total_amount'])
    
    df_monthly_product_tr = df_tr_receipt_menumap.groupby(['year_month','cate_name']).apply(aggregation)
    
    df_default = genDefaultMontlyCateTotalAmountDataFrame(df_monthly_product_tr,year)
    df_all_monatly_sales_volume = pd.merge(df_default, df_monthly_product_tr, left_index=True, right_index=True, how='outer').fillna(0).sort_index(ascending='1')

    def post_aggregation(row):
        return row['total_amount_x'] + row['total_amount_y']
    
    df_all_monatly_sales_volume['total_amount'] = df_all_monatly_sales_volume.apply(post_aggregation, axis=1)
    df_all_monatly_sales_volume.drop(['total_amount_x','total_amount_y'], axis=1, inplace=True)

    def gen_dict_total_amount(month_rows):
        monthlyDict = {}
        monthlyDictKey = month_rows.index.get_level_values('year_month')[0]
        
        monthCateItemsStr = "{"
        for item in zip(month_rows.index.get_level_values('cate_name'),month_rows['total_amount']):
            monthCateItemsStr += "'{0}':{1},".format(item[0],item[1]);
        
        monthCateItemsStr = monthCateItemsStr[:-1]
        monthCateItemsStr += "}"
        
        monthlyDict = ast.literal_eval(monthCateItemsStr)
        monthlyDict['year_month'] = month_rows.index.get_level_values('year_month')[0]

        return monthlyDict
    
    mothlyTotalAmountDictItems = df_all_monatly_sales_volume.groupby(df_all_monatly_sales_volume.index.get_level_values('year_month')).apply(gen_dict_total_amount)    
 
    mothlyTotalAmountDict = {}
    mothlyTotalAmountList = []
    for item in mothlyTotalAmountDictItems:
        mothlyTotalAmountList.append(item)
    mothlyTotalAmountDict['total_amount'] = mothlyTotalAmountList

    return mothlyTotalAmountDict

result = agg_montly_total_amount_by_product_cate('2014')

NameError: global name 'connect' is not defined

In [60]:
import pandas as pd
from impala.dbapi import connect
from impala.util import as_pandas
import ast
import math
from pandas import  MultiIndex, Series

def genDefaultMontlyCateTotalAmountDataFrame(df_monthly_product_tr,year):
    unique_count_of_category = len(df_monthly_product_tr.index.get_level_values(1).unique())

    month_index_arr = []
    cate_index_arr = []

    for month in range(1,13):
        for cate in range(unique_count_of_category):
            month_index_arr.append("{0}-{1:02d}".format(year,month))

    unique_cate_index_arr = df_monthly_product_tr.index.get_level_values(1).unique()
    #df_monthly_product_tr.ix[0:unique_count_of_category].index.get_level_values(1)

    for month in range(1,13):
        for cate in unique_cate_index_arr:
            cate_index_arr.append(cate)

    full_month_cate_multi_index = pd.MultiIndex.from_tuples(zip(month_index_arr, cate_index_arr), names=['year_month', 'product_name'])
    df_full_month_cate_default = pd.DataFrame(0, index=full_month_cate_multi_index, columns=['total_amount'])
    return df_full_month_cate_default

def gen_dict_total_amount(row):
    
    monthlyDict = {}
    monthlyDictStr = "{"
    for key,value in zip(row.index, row): 
        monthlyDictStr += "'{0}':{1},".format(key,value)
    
    monthlyDictStr = monthlyDictStr[:-1]
    monthlyDictStr += "}"
    
    monthlyDict = ast.literal_eval(monthlyDictStr)
    monthlyDict['year_month'] = row.name
 
    return monthlyDict


def agg_montly_total_amount_by_product(year, product_cate):
    
    conn = connect(host='salest-master-server', port=21050)
    cur = conn.cursor()

    cur.execute('USE salest')
    cur.execute(
        """
        SELECT * FROM (
            SELECT SUBSTR(view_tr_receipt.date_receipt_num,1,7) AS year_month, 
                  view_tr_receipt.num_of_product, view_tr_receipt.sales_amount AS total_amount,
                  ext_menumap_info.product_name, ext_menumap_info.cate_name, ext_menumap_info.price
            FROM (SELECT * FROM ext_tr_receipt WHERE SUBSTR(date_receipt_num,1,4) = '""" + year + "'" +
            """) view_tr_receipt JOIN ext_menumap_info USING (product_code)
        ) view_tr_receipt_menumap
        WHERE cate_name = '커피'
        """
        #""" + "'" + product_cate.encode('utf8') + "'"
    )
    
    df_monthly_product_tr = as_pandas(cur)
    conn.close()
        
    column_func_tuple = [('total_amount','sum')]
    df_monthly_summary = df_monthly_product_tr.groupby(['year_month','product_name'])['total_amount'].agg(column_func_tuple)
    df_monthly_summary.rename(columns={'total_amount': 'total_amount_B'}, inplace=True)

    df_default = genDefaultMontlyCateTotalAmountDataFrame(df_monthly_summary,year)
    df_default.rename(columns={'total_amount': 'total_amount_A'}, inplace=True)

    df_per_category = pd.concat([df_default, df_monthly_summary], axis=1).fillna(0)

    def post_aggregation(row):
        return row[0] + row[1]
    
    df_per_category['total_amount'] = df_per_category.apply(post_aggregation, axis=1)
    df_per_category.drop(['total_amount_A','total_amount_B'],axis=1,inplace=True)
  
     # Overall Top 10 menu items in category 
    
    df_topten_products_by_total_amount = df_monthly_product_tr.groupby(['product_name']).sum().sort_values(by='total_amount', ascending=False)[:10]
    df_topten_products_by_total_amount.drop(['num_of_product'],axis=1, inplace=True)
    df_topten_products_by_total_amount.rename(columns={'total_amount':'overall_total_amount'},inplace=True)

    # Merge the above two dataframes
    df_new = df_per_category.reset_index(level=0)
    df_merged = pd.merge(df_new, df_topten_products_by_total_amount, left_index=True, right_index=True, how='left').sort_values(by='year_month', ascending=True)
    
    def agg_monthly_items_summary(row):
        sr_columns = row[row['overall_total_amount'].notnull()].index
        sr_values = row[row['overall_total_amount'].notnull()]['total_amount']

        etcSum = row[row['overall_total_amount'].isnull()]['total_amount'].sum()

        sr_columns = sr_columns.insert(sr_columns.size,'ETC')
        sr_etc = pd.Series([etcSum], index=['ETC'])
        sr_values = sr_values.append(sr_etc)

        return pd.Series(sr_values, index=sr_columns)
    
    df_merged_new = df_merged.reset_index(level=0)

    df_agg_monthly_summary = df_merged.groupby(['year_month']).apply(agg_monthly_items_summary)#.unstack()
    df_agg_monthly_summary.fillna(0,inplace=True)
    
    monthlyDictItems = df_agg_monthly_summary.apply(gen_dict_total_amount,axis=1)
    
    mothlyTotalAmountDict = {}
    mothlyTotalAmountList = []
    for item in monthlyDictItems:
        mothlyTotalAmountList.append(item)
    mothlyTotalAmountDict['total_amount'] = mothlyTotalAmountList
    
    return mothlyTotalAmountDict

agg_montly_total_amount_by_product('2015', '커피')


{'total_amount': [{'ETC': 420700.0,
   'year_month': '2015-01',
   '\xec\x95\x84\xeb\xa9\x94\xeb\xa6\xac\xec\xb9\xb4\xeb\x85\xb8': 1057500.0,
   '\xec\x95\x84\xec\x9d\xb4\xec\x8a\xa4\xeb\xb0\x94\xeb\x8b\x90\xeb\x9d\xbc\xeb\x9d\xbc\xeb\x96\xbc': 28000.0,
   '\xec\x95\x84\xec\x9d\xb4\xec\x8a\xa4\xec\x95\x84\xeb\xa9\x94\xeb\xa6\xac\xec\xb9\xb4\xeb\x85\xb8': 310000.0,
   '\xec\x95\x84\xec\x9d\xb4\xec\x8a\xa4\xec\x95\x84\xeb\xa9\x94\xeb\xa6\xac\xec\xb9\xb4\xeb\x85\xb8UP': 0.0,
   '\xec\x95\x84\xec\x9d\xb4\xec\x8a\xa4\xec\xb9\xb4\xeb\x9d\xbc\xeb\xa9\x9c\xeb\xa7\x88\xeb\x81\xbc\xec\x95\xbc\xeb\x98\x90': 39600.0,
   '\xec\x95\x84\xec\x9d\xb4\xec\x8a\xa4\xec\xb9\xb4\xed\x8e\x98\xeb\x9d\xbc\xeb\x96\xbc': 18000.0,
   '\xec\x95\x84\xec\x9d\xb4\xec\x8a\xa4\xec\xb9\xb4\xed\x8e\x98\xeb\xaa\xa8\xec\xb9\xb4': 51800.0,
   '\xec\xb9\xb4\xeb\x9d\xbc\xeb\xa9\x9c\xeb\xa7\x88\xeb\x81\xbc\xec\x95\xbc\xeb\x98\x90': 176400.0,
   '\xec\xb9\xb4\xed\x8e\x98\xeb\x9d\xbc\xeb\x96\xbc': 246000.0,
   '\xec\xb9\xb4\xed\